In [14]:
import numpy as np
import pandas as pd

import umap
import hdbscan
import pprint

from sentence_transformers import SentenceTransformer
import torch

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('punkt')
from nltk import tokenize

[nltk_data] Downloading package punkt to /home1/kasee0/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
import transformers

In [16]:
import pickle
with open('./filtered_Main.pkl', 'rb') as f:
    data1=pickle.load(f)
with open('./filtered_Methods.pkl', 'rb') as f2:
    data2=pickle.load(f2)
with open('./filtered_Sum.pkl', 'rb') as f3:
    data3=pickle.load(f3)

In [20]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

In [ ]:
model_name='allenai/scibert_scivocab_uncased'

auto_tokenizer = AutoTokenizer.from_pretrained(model_name)
auto_model= AutoModel.from_pretrained(model_name)

In [33]:
text_as_string=str(data1[0].split('.')[14])
print(text_as_string)

 The EDS spectra correlates with positions 1, 2, 3, 4, and 5, as shown in Figure 2c, which confirms the presence of Cr, Gd, Ti, O, and N elements in the respective layers


In [34]:
auto_tokens = auto_tokenizer.tokenize(text_as_string)

In [35]:
auto_tokens

['the',
 'eds',
 'spectra',
 'correlates',
 'with',
 'positions',
 '1',
 ',',
 '2',
 ',',
 '3',
 ',',
 '4',
 ',',
 'and',
 '5',
 ',',
 'as',
 'shown',
 'in',
 'figure',
 '2',
 '##c',
 ',',
 'which',
 'confirms',
 'the',
 'presence',
 'of',
 'cr',
 ',',
 'gd',
 ',',
 'ti',
 ',',
 'o',
 ',',
 'and',
 'n',
 'elements',
 'in',
 'the',
 'respective',
 'layers']

In [37]:
from sentence_transformers import SentenceTransformer, models
from torch import nn

In [39]:
word_embedding_model = models.Transformer('allenai/scibert_scivocab_uncased', max_seq_length=512)

loading configuration file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/config.json from cache at /home1/kasee0/.cache/huggingface/transformers/858852fd2471ce39075378592ddc87f5a6551e64c6825d1b92c8dab9318e0fc3.03ff9e9f998b9a9d40647a2148a202e3fb3d568dc0f170dda9dda194bab4d5dd
Model config BertConfig {
  "_name_or_path": "allenai/scibert_scivocab_uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31090
}

loading weights file https://huggingface.co/allenai/scibert_scivocab_uncased/resolve/main/pytorch_model.bin f

In [ ]:
# Sentence transformer를 사용해서 Word embedding model을 sentence embedding model로 만드는 방법

pooling_model=models.Pooling(word_embedding_model.get_word_embedding_dimension(),
pooling_mode_max_tokens=False,
Pooling_mode_mean_tokens=False,
Pooling_mode_mean_sqrt_len_token=True)

dense_model= models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(),out_features=768,activation_function=nn.Tanh())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model) #, dense_model])


In [ ]:
# Setence transformer를 사용하지 않고 word embedding model을 sentence embedding model로 변환하는 방법

from transformers import AutoTokenizer,AutoModel
import torch

# mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

# setences we want sentece embeddings for
sentences = ["I love you", "I hate you", "I love my dog"]

#Loda automodel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

# Tokenize senteces
encoded_input= tokenizer(sentences, padding=True, truncation = True, max_length=512, return_tensors='pt')

with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling.
setence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
# Sentence transformer를 사용해서 Word embedding model을 sentence embedding model로 만드는 방법
# encoding

%%time # 시간

embeddings = model.encode(setntences, batch_size = 128)

In [ ]:
%% time

umap_neighbors = 12 # local 값 숫자를 줄일 수록 local에 집중
umap_n_components = 12

umap_embeddings_12d = umap.UMAP(n_neigbors=umap_neighbors,
                                n_componets=umap_n_components
                                n_epochs=14000,
                                min_dist=0.1,
                                low_memory=False,
                                learning_rate=0.5
                                verbose=True,
                                metric='cosine',
                                spread=3.0,
                                local_connectivity=2,
                                target_metric='l2').fit_transform(embeddings)

In [ ]:
%% time

hdbscan_minimum_cluster_size = 55
hdbscan_min_samples = 2

cluster = hdbscan.HDBSCAN(min_cluster_size=hdbscan_minimum_cluster_size,
                          min_samples=hdbscan_min_samples,
                          metric='euclidean',
                          cluster_selection_epsilon=0.1,
                          cluster_selection_method='leaf',
                          leaf_size=40,
                          algorithm='best').fit(umap_embeddings_12d)

analyze_clusters= len(pd.Series(cluster.labels_).unique())

df = pd.DataFrame(pd.DataFrame(cluster.labels_).value_counts())

with pd.option_context('display.max_rows', 30, 'display.min_rows', 10, 'display,max_colums',10, 'display.width', 100):
    print(df.head(20))

# 3D visualization

In [ ]:
%% time

umap_neighbors = 12 # local 값 숫자를 줄일 수록 local에 집중
umap_n_components = 3

umap_data_3d = umap.UMAP(n_neigbors=umap_neighbors,
                         n_componets=umap_n_components,
                         n_epochs=14000,
                         min_dist=0.1,
                         low_memory=False,
                         learning_rate=0.1
                         verbose=True,
                         metric='cosine',
                         spread=3.0,
                         local_connectivity=2,
                         target_metric='l2').fit_transform(embeddings)

In [ ]:
result3d = pd.DataFrame(umap_data_3d, colums=['x', 'y', 'z'])
result3d['sentece'] = sentences
result3d['labels'] = cluster.labels_

with pd.option_context('display.max_rows', 30, 'display.min_rows', 10, 'display,max_colums',10, 'display.width', 200,'display.max_colwidth', 160):
    print(result3d.head(20))

## Bigrams and trigrams in each cluster

In [40]:
vectorizer = CountVectorizer(ngram_range=(2,3), stop_words='english')
count = vectorizer.fit_transfrom(docs_per_topic.Doc.values.tolist())

t = count.toarray()
w = t.sum(axis=1)

tf = np.divide(t.T, w)
sum_t = t.sum(axis=0)

m=len(sentences)
idf = np.log(np.divide(m,sum_t)).reshape(-1,1)

tf_idf = np.multiply(tf,idf)
tf_idf = np.array(tf_idf, dtype=np.float32)


AttributeError: 'CountVectorizer' object has no attribute 'fit_transfrom'

In [ ]:
n = 8

words = vectorizer.get_feature_names()
labels = list(docs_per_topic.Topcic)

tf_idf_transposed = tf_idf.T
indices = tf_idf_transposed.argsort()[:, -n:]

tf = []
top_n_words = {}

def Sort_Tuple(tup):
    tup.sort(key=lambda x: x[1])
    return tup

for i, label in enumerate(labels):
    


In [ ]:
from unittest import result


outliers3d = result3d.loc[result3d.labels == -1, :]
clustered3d = result3d.loc[result3d.labels != -1, :]

final_df = pd.merge(clustered3d, new_df2, left_on = ['labels'],right_on=['labels'])

final_df_noise = pd.merge(outliers3d, new_df2, left_on = ['labels'],right_on=['labels'])

final_df_sorted_labels = final_df.sort_values(by=['labels'], right_on = ['labels'])

final_df_sorted_labels = final_df.sort_values(by=['labels'], ascending=True)

my_final_df_sorted_labels_8 = final_df_sorted_labels.loc[final_df_sorted_labels['labels']==2]

with pd.option_context('display.max_rows', 30, 'display.min_rows', 10, 'display,max_colums',10, 'display.width', 200,'display.max_colwidth', 160):
    print(my_final_df_sorted_labels_8.head(20))

# Bigrams of the documents

In [ ]:
from collections import Counter

class my_ngrams:
    def __init__(self,words_list, dim_n):
        self.words_lsit = words_list
        self.dim_n = dim_n

    def generate_ngram(self):
        ngrams = zip(*[self.words_list[i:] for i in range(self.dim_n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngrams_counter(self):
        my_ngrams_counter = Counter(self.generate_ngram())
        most_common_my_ngram = my_ngrams_counter.most_common()
        most_common_my_ngram = pd.DataFrame(most_common_my_ngram)
        most_common_my_ngram.colums = ['word','freq']
        most_common_my_ngram['percentage'] = most_common_my_ngram['freq'] *100 / sum(most_common_my_ngram.freq)
        return most_common_my_ngram

for t in[0,1,2,3]:
    my_words2 = final_df_sorted_labels.sentence.loc[final_df_sorted_labels['labels']==t].apply(lambda x: x.split()).tolist()
    my_words_list2 = [item for sublist in my_words2 for item in sublist]
    my_words_counter2 = Counter(my_words_list2)
    gla = my_ngrams(my_words_list2, nn).ngrams_couter()

my_words2 = final_df_sorted_labels.stentece.apply(lambda x: x.split()).tolist()
my_words_list2 = [item for sublist in my_words2 for item in sublist]
my_words_counter2 = Counter(my_words_list2)
nn=3
gla = my_ngrams(my_words_list2, nn).ngrams_counter()
gla.head(12).style.format({'percentage': "{:.3f}%"}).bar(color='#FFA07A', vim=10, subset=['freq'], align='zero').highlight_max(axis=0,color='lightgreen'))

In [41]:
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.io as pio
pio.renderers
pio.renderers.default = 'iframe'

In [ ]:
fig = go.Figure(go.Scatter3d(x=final_df_nmoise.x,
                             y=final_df_nmoise.y,
                             z=final_df_nmoise.z,
                             mode='text+markers',
                             text=final_df_noise.labels,
                             textfont=dict(size=9,family="Arial",color ='rgba(128,128,128,0.0)'),
                             hoverinfo='text',
                             hoberterxt= final_df_noise.sentece,
                             name = 'NOISE - no activ cluster allocation of these senteces',
                             textposition='middle right',
                             marker=dict(symbol='circle',
                                         sizemin=13,
                                         size=np.array(final_df_noise.magnitued),
                                         sizeref=1400,
                                         sizemode = 'diameter',
                                         color='lightgray'),showlegend=True)
                                         )